<a href="https://colab.research.google.com/github/AndyRids/AndyRids/blob/main/TRAJECTORY_OUTLIERS.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [138]:
import json
import math
from typing import Union

import ipyleaflet as leaf
import ipywidgets as widgets
import matplotlib.pyplot as plt
import numpy as np
from geopandas import GeoDataFrame, points_from_xy
from pandas import DataFrame, Timestamp, offsets
from pandas.api.indexers import (
    FixedForwardWindowIndexer, VariableOffsetWindowIndexer)
from scipy import stats
from scipy.spatial.distance import cdist, euclidean
from shapely import LineString

In [126]:
%matplotlib inline

In [3]:
np.random.seed(2017)

In [165]:
X = (
    GeoDataFrame(
        data=np.array(
            [
                [52.24947, 0.16015],
                [52.24938, 0.16001],
                [52.2493, 0.15992],
                [52.2492, 0.15982],
                [52.24913, 0.15978],
                [52.24908, 0.15982],
                [52.24907, 0.15987],
                [52.24901, 0.15994],
                [52.24897, 0.16003],
                [52.2492, 0.16068],
                [52.24886, 0.16025],
                [52.2488, 0.16037],
                [52.24878, 0.16036],
                [52.24873, 0.16035],
                [52.24869, 0.1603],
                [52.24864, 0.16027],
                [52.24862, 0.16024],
                [52.24828, 0.16017],
            ]
        ),
        columns=["LAT_DD", "LONG_DD"],
    )
    .assign(
        GEOMETRY=lambda x: points_from_xy(x.LONG_DD, x.LAT_DD, crs="EPSG:4326")
    )
    .set_geometry("GEOMETRY")
    .assign(
        DIST_NEXT_M=lambda x: (
            x.to_crs("EPSG:4978").distance(x.to_crs("EPSG:4978").shift()))
    )
)

X

,LAT_DD,LONG_DD,GEOMETRY,DIST_NEXT_M
0,52.24947,0.16015,POINT (0.16015 52.24947),NaN
1,52.24938,0.16001,POINT (0.16001 52.24938),12.414489
2,52.24930,0.15992,POINT (0.15992 52.24930),9.344573
3,52.24920,0.15982,POINT (0.15982 52.24920),11.137752
4,52.24913,0.15978,POINT (0.15978 52.24913),6.737352
5,52.24908,0.15982,POINT (0.15982 52.24908),5.178267
6,52.24907,0.15987,POINT (0.15987 52.24907),3.526328
7,52.24901,0.15994,POINT (0.15994 52.24901),7.121904
8,52.24897,0.16003,POINT (0.16003 52.24897),7.082828
9,52.24920,0.16068,POINT (0.16068 52.24920),48.787015


In [216]:
def geometric_median(
        X: Union[DataFrame, GeoDataFrame],
        eps=1e-5):
    """Implementation of Yehuda Vardi and Cun-Hui Zhang's algorithm for the
    geometric median. Weights have not been implimented.

    Citation:
        Vardi†, Y. & Zhang, C. (2000) The multivariate L1-median and
        associated data depth
    """

    # initialising 'median' to the centroid
    y = np.mean(X, 0)

    while True:
        D = cdist(X, [y])
        nonzeros = (D != 0)[:, 0]

        Dinv = 1 / D[nonzeros]
        Dinvs = np.sum(Dinv)
        W = Dinv / Dinvs
        T = np.sum(W * X[nonzeros], 0)

        num_zeros = len(X) - np.sum(nonzeros)
        if num_zeros == 0:
            y1 = T
        elif num_zeros == len(X):
            return y
        else:
            R = (T - y) * Dinvs
            r = np.linalg.norm(R)
            rinv = 0 if r == 0 else num_zeros/r
            y1 = max(0, 1-rinv)*T + min(1, rinv)*y

        if euclidean(y, y1) < eps:
            return y1

        y = y1

In [ ]:
# FixedForwardWindowIndexer VariableOffsetWindowIndexer

In [234]:
def trajectory_zscore(
        X: Union[DataFrame, GeoDataFrame],
        window_size: int = 5,
        std_threshold: float = 1.5) -> Union[DataFrame, GeoDataFrame]:
    """Calculates the Z-Score of a trajectory DataFrame latitude & longitude
    values, using a sliding window and assigns a SW_NOISE column indicating
    whether the window centre point's Z-Score is >= the threshold.

    Info:
        - Z-Score is a statistical measurement of a score's relationship
        to the mean in a group of scores

        - Can reveal if a value is typical/atypical for a specified data set

        - In general, a Z-score of -3.0 to 3.0 suggests that a value is
        within three standard deviations of its mean

    Formula:
        z = ( x - μ ) / σ

    Parameters:
        X (Union[DataFrame, GeoDataFrame]): Trajectory data

        window_size (int): Sliding window size

        std_threshold (float): Zscore threshold

    Returns:
        (Union[DataFrame, GeoDataFrame])
    """

    # must specify columns in case of GeoDataFrame
    # as operators are used for geospatial functions
    columns = ["LAT_DD", "LONG_DD"]

    # sliding window, with min_periods of 1 and center=True
    # ensures all points at all indices receive a zscore
    Xr = (
        X[["LAT_DD", "LONG_DD"]]
            .rolling(window=window_size, min_periods=1, center=True))

    # calculate absolute zscore value
    Xz = ((X[["LAT_DD", "LONG_DD"]] - Xr.mean()) / Xr.std(ddof=1)).abs()

    # for the indices where Xz.LAT_DD >= std_threshold is False, the results
    # of Xz.LONG_DD >= std_threshold are returned, which can be True|False.
    # A True value indicates an outlier latitude or longitude point
    return X.assign(
        SW_NOISE=np.where(
            # where condition == True...
            Xz.LAT_DD >= std_threshold,
            # return True...
            True,
            # else value of next condition...
            Xz.LONG_DD >= std_threshold
        )
    )

def trajectory_modified_zscore(
        X: Union[DataFrame, GeoDataFrame],
        window_size: int = 5,
        mad_sigma: float = 3.5) -> Union[DataFrame, GeoDataFrame]:
    """Calculates the Median Absolute Deviation (MAD) of a trajectory
    DataFrame latitude & longitude values, using a sliding window and
    assigns a SW_NOISE column indicating whether the window centre
    point's MAD is >= the threshold.

    Info:
        - MAD is a robust measure of staistical dispersion, being less
        sensitive to outliers compared to the standard deviation

        - Sensitivity to outliers can be adjusted by changing the
        multiplier used to calculate its deviation-like measure

        - Particularly useful for time series data

        - If more than 50% of the distribution elements are zero, MAD
        will also be zero

    Formula:
        MAD = C x median(|X − median(X)|)

        C is a constant scale factor. This metric can be used as a robust
        alternative to the standard deviation.

    Parameters:
        X (Union[DataFrame, GeoDataFrame]): Trajectory data

        window_size (int): Sliding window size

        mad_threshold (float): MAD threshold

    Returns:
        (Union[DataFrame, GeoDataFrame]): X with SW_NOISE noise column
        indicating outliers (True|False) in LAT_DD|LONG_DD
    """

    # must specify columns in case of GeoDataFrame
    # as operators are used for geospatial functions
    columns = ["LAT_DD", "LONG_DD"]

    # sliding window, with min_periods of 1 and center=True
    # ensures all points at all indices receive a zscore
    Xr = (
        X[["LAT_DD", "LONG_DD"]]
            .rolling(window=window_size, min_periods=1, center=True))

    # calculate the MAD value and then the sliding window median
    medianAD = (
        X[["LAT_DD", "LONG_DD"]]
            .subtract(Xr.median())
            .abs()
            .rolling(window=window_size, min_periods=1, center=True)
            .median()
    )

    display(Xr.std(ddof=1) / medianAD)

    # calculate the Mean Absolute Deviation
    meanAD = (
        X[["LAT_DD", "LONG_DD"]]
            .subtract(Xr.mean())
            .abs()
            .rolling(window=window_size, min_periods=1, center=True)
            .mean()
    )

    # MAD becomes 0 if > 50% distribution elements are == 0
    # Where MAD == 0, subtract the median from the score and
    # divide by 1.253314 * MeanAD. 1.253314 * MeanAD is
    # approximately equal to the standard deviation

    # where MAD is not 0, subtract the median from the score and
    # divide by 1.486 * MAD. 1.486 * MAD approximately equals the
    # standard deviation

    # 1.4826 value is used as a scale factor, because it is the
    # inverse of the 0.75 quantile for the standard normal distribution
    # at this quantile, 50% of the standard normal CDF is covered
    MAD = np.where(
        # where medianAD == 0...
        medianAD == 0.0,
        # return MeanAD * 1.253314 (~std)
        meanAD * 1.253314,
        # else return medianAD * 1.4826 (~std)
        medianAD * 1.4826
    )

    Xz = ((X[["LAT_DD", "LONG_DD"]] - Xr.median()).abs()) / MAD

    # for the indices where Xt.LAT_DD >= std_threshold is False, the results
    # of Xt.LONG_DD >= std_threshold are returned, which can be True or False
    # a True value indicates an outlier latitude or longitude point
    return X.assign(
        SW_NOISE=np.where(
            # where condition == True...
            Xz.LAT_DD >= mad_sigma,
            # return True...
            True,
            # else value of next condition...
            Xz.LONG_DD >= mad_sigma
        )
    )

def trajectory_geo_modified_zscore(
        X: Union[DataFrame, GeoDataFrame],
        mad_sigma: float = 2.0) -> Union[DataFrame, GeoDataFrame]:
    """Calculates the modified Z-score using the Geometric Median Absolute
    Deviation (MAD) of a trajectory latitude & longitude values.

    Formula:
        geometric median is calculated using an implementation of Yehuda
        Vardi and Cun-Hui Zhang's algorithm.

        MAD = C x median(|X − geometric median(X)|)

        C is a constant scale factor. This metric can be used as a robust
        alternative to the standard deviation.

    Parameters:
        X (Union[DataFrame, GeoDataFrame]): Trajectory data

        mad_sigma (float): Z-Score MAD threshold

    Returns:
        (Union[DataFrame, GeoDataFrame]): X with SW_NOISE noise column
        indicating outliers (True|False) in LAT_DD|LONG_DD
    """

    # must specify columns in case of GeoDataFrame as
    # operators can be used for geospatial functions
    geo_columns = ["LAT_DD", "LONG_DD"]

    geo_median = geometric_median(X[["LAT_DD", "LONG_DD"]])

    geo_medianAD = (X[["LAT_DD", "LONG_DD"]] - geo_median).abs().median()

    # constant to make MAD representative of the standard deviation
    # as the data is not normally distributed, we estimate using
    # the standard deviation / MAD
    C = X[["LAT_DD", "LONG_DD"]].std(ddof=0) / geo_medianAD


    Xz = (
        (X[["LAT_DD", "LONG_DD"]] - geo_median).abs() / (C * geo_medianAD)
    )

    return X.assign(
        SW_NOISE=np.where(
            # where condition == True...
            Xz.LAT_DD >= mad_sigma,
            # return True...
            True,
            # else value of next condition...
            Xz.LONG_DD >= mad_sigma
        )
    )

In [97]:
mu, sigma = 0, 1 # mean and standard deviation
norm_data = np.random.normal(mu, sigma, 1000)

norm_data_MedianAD = (
    # median of the absolute diference from the median
    np.median(np.abs(norm_data - np.median(norm_data)))
)


## Z-Score

Window size of 8 and htrshold of 1.5 identifies both outliers in the trajectory.

In [249]:
trajectory_zscore(X, window_size=8, std_threshold=1.5)

,LAT_DD,LONG_DD,GEOMETRY,DIST_NEXT_M,SW_NOISE
0,52.24947,0.16015,POINT (0.16015 52.24947),NaN,False
1,52.24938,0.16001,POINT (0.16001 52.24938),12.414489,False
2,52.24930,0.15992,POINT (0.15992 52.24930),9.344573,False
3,52.24920,0.15982,POINT (0.15982 52.24920),11.137752,False
4,52.24913,0.15978,POINT (0.15978 52.24913),6.737352,False
5,52.24908,0.15982,POINT (0.15982 52.24908),5.178267,False
6,52.24907,0.15987,POINT (0.15987 52.24907),3.526328,False
7,52.24901,0.15994,POINT (0.15994 52.24901),7.121904,False
8,52.24897,0.16003,POINT (0.16003 52.24897),7.082828,False
9,52.24920,0.16068,POINT (0.16068 52.24920),48.787015,True


In [240]:
def style_fn(x):
    outlier = (
        {"fillColor": "red", "weight": 2, "radius": 10}
        if x["properties"]["SW_NOISE"]
        else dict()
    )

    return outlier

In [241]:
m_1 = leaf.Map(
    center=[52.24886, 0.16025],
    zoom=18,
    layout=widgets.Layout(height="500px", width="400px"),
    scroll_wheel_zoom=True,
)

m_1.add(leaf.GeoJSON(
    data=json.loads(trajectory_zscore(X, window_size=8, std_threshold=1.5).to_json()),

    style_callback=style_fn,
    point_style={
        "color": "black",
        "fillColor": "blue",
        "fillOpacity": 0.7,
        "radius": 7,
        "weight": 1,
    }
)

)
m_1.add(
    leaf.Polyline(
        locations=[
            sorted(xy, reverse=True)
            for xy in np.stack(LineString(X.GEOMETRY).xy, axis=1)
        ],
        fill=False
    )
)

Map(center=[52.24886, 0.16025], controls=(ZoomControl(options=['position', 'zoom_in_text', 'zoom_in_title', 'z…

## Modified Z-Score With a Sliding Window

Window size set to 6 and threshold of at least 2.5, identifies the the midal and final points as outliers.

In [248]:
m_2 = leaf.Map(
    center=[52.24886, 0.16025],
    zoom=18,
    layout=widgets.Layout(height="500px", width="400px"),
    scroll_wheel_zoom=True,
)

m_2.add(leaf.GeoJSON(
    data=json.loads(
        trajectory_modified_zscore(X, window_size=6, mad_sigma=2.5)
            .to_json()),
    style_callback=style_fn,
    point_style={
        "color": "black",
        "fillColor": "blue",
        "fillOpacity": 0.7,
        "radius": 7,
        "weight": 1,
    }
)

)
m_2.add(
    leaf.Polyline(
        locations=[
            sorted(xy, reverse=True)
            for xy in np.stack(LineString(X.GEOMETRY).xy, axis=1)
        ],
        fill=False
    )
)

,LAT_DD,LONG_DD
0,2.126225,2.575606
1,2.555556,2.949874
2,3.403124,2.988645
3,4.008879,2.985221
4,4.172219,2.397277
5,3.473178,2.091429
6,1.938382,2.327373
7,1.647625,8.383019
8,1.913161,8.067553
9,2.405434,7.637081


Map(center=[52.24886, 0.16025], controls=(ZoomControl(options=['position', 'zoom_in_text', 'zoom_in_title', 'z…

## Modified Z-Score With Geometric Median

Identifies all points as outliers with threshold of 2. Cannot use a sliding window and must identify outliers in the context of the whole trajectory, due to the nature of calculating the Geometric Median.

In [245]:
m_3 = leaf.Map(
    center=[52.24886, 0.16025],
    zoom=18,
    layout=widgets.Layout(height="500px", width="400px"),
    scroll_wheel_zoom=True,
)

m_3.add(leaf.GeoJSON(
    data=json.loads(
        trajectory_geo_modified_zscore(X, mad_sigma=2.0)
            .to_json()),
    style_callback=style_fn,
    point_style={
        "color": "black",
        "fillColor": "blue",
        "fillOpacity": 0.7,
        "radius": 7,
        "weight": 1,
    }
))

m_3.add(
    leaf.Polyline(
        locations=[
            sorted(xy, reverse=True)
            for xy in np.stack(LineString(X.GEOMETRY).xy, axis=1)
        ],
        fill=False
    )
)

Map(center=[52.24886, 0.16025], controls=(ZoomControl(options=['position', 'zoom_in_text', 'zoom_in_title', 'z…